In [1]:
import numpy as np 

from qiskit_optimization import QuadraticProgram
from qiskit_optimization.converters import QuadraticProgramToQubo
from qiskit_optimization.algorithms import CplexOptimizer
from qiskit import Aer
from qiskit.primitives import Sampler, BackendSampler
from qiskit.circuit.library import TwoLocal

import optimizer as opt

In [2]:
target_mdl = QuadraticProgram()
target_mdl.read_from_lp_file('instances/Random_seq2_c.lp')

N = target_mdl.get_num_vars()
print(N)

#print(target_mdl.prettyprint())

2


In [3]:
cplex_result = CplexOptimizer().solve(target_mdl)
cplex_optval = cplex_result.fval
cplex_xsol = cplex_result.x.astype(int)

print(
        '-'*110,
        '[CPLEX Result]',
        f'   optimal value: {cplex_optval:.3f}',
        f'   optimal solution (binary): {cplex_xsol}',
        f'   status: {cplex_result.status.name}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[CPLEX Result]
   optimal value: -3.400
   optimal solution (binary): [1 0]
   status: SUCCESS
--------------------------------------------------------------------------------------------------------------


In [4]:
qubo = QuadraticProgramToQubo().convert(target_mdl)

In [5]:
num_qubits = N
print(f'number of system qubits: {num_qubits}')

number of system qubits: 2


In [6]:
reps = 2

ansatz = TwoLocal(
    num_qubits = num_qubits,
    rotation_blocks = 'ry', 
    entanglement_blocks = 'cz',
    entanglement = 'pairwise', 
    reps = reps,
    insert_barriers = True
    )

n_params = ansatz.num_parameters
print(f'number of params: {n_params}')

number of params: 6


In [7]:
shots = 2**10
backend = Aer.get_backend('aer_simulator_matrix_product_state')

sampler = BackendSampler(backend=backend)
sampler.set_options(shots=shots)

print(f'shots = {shots}')

shots = 1024


In [8]:
def objective(x):
    objval = qubo.objective.evaluate(x)
    return objval

In [9]:
print(f'reference value: {cplex_optval:.3f}')
print('-'*80)

QuantumOptimizer = opt.Quantum_Optimizer()
QuantumOptimizer.set_optimizer(num_qubits, ansatz, sampler)

theta_initial = np.hstack([np.full(num_qubits, np.pi/2), np.zeros(num_qubits*ansatz.reps)])

result = QuantumOptimizer.run(
    objective = objective, 
    qc_params_init = theta_initial,
    max_epoch = 5, 
    random_update = True, 
    intermediate_result_display = True,
    refval = None,
    alpha = 0.1 #cVAR param
)

reference value: -3.400
--------------------------------------------------------------------------------
epoch = 0 	 step = 0 	 best val = -3.400 	 expectation = -3.400
epoch = 0 	 step = 1 	 best val = -3.400 	 expectation = -3.400
epoch = 0 	 step = 2 	 best val = -3.400 	 expectation = -3.400
epoch = 0 	 step = 3 	 best val = -3.400 	 expectation = -3.400
epoch = 0 	 step = 4 	 best val = -3.400 	 expectation = -3.400
epoch = 0 	 step = 5 	 best val = -3.400 	 expectation = -3.400
final: 	 best val = -3.400
epoch = 1 	 step = 0 	 best val = -3.400 	 expectation = -3.400
epoch = 1 	 step = 1 	 best val = -3.400 	 expectation = -3.400
epoch = 1 	 step = 2 	 best val = -3.400 	 expectation = -3.400
epoch = 1 	 step = 3 	 best val = -3.400 	 expectation = -3.400
epoch = 1 	 step = 4 	 best val = -3.400 	 expectation = -3.400
epoch = 1 	 step = 5 	 best val = -3.400 	 expectation = -3.400
final: 	 best val = -3.400
epoch = 2 	 step = 0 	 best val = -3.400 	 expectation = -3.400
epoch = 2

In [10]:
x = QuantumOptimizer.best_x
objval = qubo.objective.evaluate(x)

print(
        '-'*110,
        '[Quantum Result]',
        f'   objective:   {objval:.3f}',
        f'   solution:    {x}',
        f'   is feasible: {target_mdl.is_feasible(x)}',
        '-'*110,
        sep='\n'
    )

--------------------------------------------------------------------------------------------------------------
[Quantum Result]
   objective:   -3.400
   solution:    [1 0]
   is feasible: True
--------------------------------------------------------------------------------------------------------------


In [11]:
result['theta']

array([17.27875959, 17.27875959, 13.35176878, 15.70796327, 15.70796327,
       15.70796327])